In [ ]:
import datetime
import matplotlib.pyplot as plt
import numpy as np

from src.data_handler import DataHandler
from src.analysis_logic import AnalysisLogic
from parameters import QudiHiraParameters

# Create instance of DataHandler
- Give it the measurement folder name
- The measurement folder must be located under the data folder specified in parameters.py (eg. `Z:/Data/20220621_FR0612-F2-2S6_uhv`)

In [ ]:
data_handler = DataHandler(measurement_folder="20220621_FR0612-F2-2S6_uhv", params=QudiHiraParameters)

# Plot all confocal images for a given measurement
- Automatically walk through measurement folders and extract a list of confocal images.
- Each list element is an instance of `MeasurementDataclass` with attributes like `data`, `params` etc.
- For additional info stored in the filename, like height, power etc., call `get_param_from_filename(unit)` with the unit of the data. e.g. the call for "20220501_confocal_50mW" will be `get_param_from_filename(unit="mW")`.
- Output folder is automatically created from the value given to `Parameters`. For ease of use in presentations/publications etc. images are saved as `.png`, `.pdf`, `.svg` and `.jpg`.

In [ ]:
confocal_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Confocal")

# Set up matplotlib figure
fig, ax = plt.subplots(nrows=len(confocal_list))

# Loop over all confocal images
for idx, confocal in enumerate(confocal_list):
    # Plot each confocal image on a subplot row
    ax[idx].imshow(confocal.data)
    # Extract the power param from the name of file
    power = confocal.get_param_from_filename(unit="mW")
    ax[idx].set_title(f"Laser power = {power}")

# Save output image
data_handler.save_figures(fig, filename="compare_confocals_at different_laser_powers")

## Filter out specific confocal images to plot
Since the confocal images are stored in the list, the required images can just be indexed out

In [ ]:
# Filter out the confocal images from index 10 to 20
filtered_list = confocal_list[10:20]

Alternatively, any of the attributes can be used to filter images

In [ ]:
indexes_to_filter = []
for idx, confocal in enumerate(confocal_list):
    # Filter based on image counts, filter all confocal images with maximum counts above 100,000
    if np.max(confocal.data) > int(1e6):
        indexes_to_filter.append(idx)
    # Filter based on laser power, filter all confocal images with laser powers between 25 and 75 mW
    if 25 < confocal.get_param_from_filename(unit="mW") < 75:
        indexes_to_filter.append(idx)
    # Filter based on timestamp, filter all confocal images taken in May 2022
    if datetime.datetime(2022, 5, 1) < confocal.timestamp < datetime.datetime(2022, 6, 1):
        indexes_to_filter.append(idx)

filtered_list = [confocal_list[idx] for idx in indexes_to_filter]

To reference a specific confocal image or a specific set of confocal images

In [ ]:
# via index
specific_confocal = confocal_list[10]

# via timestamp
specific_confocal = [confocal.timestamp.date() == datetime.datetime(2022, 5, 1) for confocal in confocal_list]

# via set of timestamps
timestamps = [datetime.datetime(2022, 5, 1, 10, 30), datetime.datetime(2022, 5, 1, 10, 35), datetime.datetime(2022, 6, 1, 12, 15)]
specific_confocals = [confocal.timestamp() == any(timestamps) for confocal in confocal_list]

# Plot all Rabi oscillations for a given measurement and fit each one to an exponentially decaying sinusoid

In [ ]:
rabi_list = data_handler.load_measurements_into_dataclass_list(measurement_str="Rabi")
analysis = AnalysisLogic()

fig, ax = plt.subplots(nrows=len(rabi_list))

for idx, rabi in enumerate(rabi_list):
    x = rabi.pulsed.measurement.data["t(ns)"]
    y = rabi.pulsed.measurement.data["spin_state"]

    # Fit data to an exponentially decaying sinusoid
    fit_x, fit_y, model = analysis.perform_fit(x, y, fit_function="sineexponentialdecay")
    
    ax[idx].plot(x, y)
    ax[idx].plot(fit_x, fit_y)
    
    # Extract the power param from the name of file
    power = rabi.get_param_from_filename(unit="dBm")
    
    # Title plot with power and T1rho time
    t1rho = model.best_fit.params["decay"]
    ax[idx].set_title(f"Power = {power}, T1rho = {t1rho}")

# Save output image
data_handler.save_figures(fig, filename="compare_rabis_at different_powers")

## Get a fit report

In [ ]:
rabi = rabi_list[5]

x, y = rabi.pulsed.measurement.data["t(ns)"], rabi.pulsed.measurement.data["spin_state"]

fit_x, fit_y, model = analysis.perform_fit(x, y, fit_function="sineexponentialdecay")

model.fit_report()

# Recompute spin state with different parameters

In [ ]:
rabi = rabi_list[5]

x, timetrace = rabi.pulsed.timetrace.data["t(ns)"], rabi.pulsed.timetrace.data["counts"]

# Analyze using mean
y = analysis.analyse_mean(timetrace, signal_start=150e-9, signal_end=400e-9)

# Analyze using mean normalized
y = analysis.analyse_mean_norm(timetrace, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)

# Analyze using mean reference
y = analysis.analyse_mean_reference(timetrace, signal_start=150e-9, signal_end=400e-9, norm_start=2000e-9, norm_end=3000e-9)